API Stockholm

In [51]:
import requests
import json
from IPython.display import display, JSON


country_code = "SE"
city = "Stockholm"
location_ids = ["2163258", "6229", "270749", "7947"]   #Torkel, Svea, sundbyberg, Stockholm E4/E20 Lil

# API key
api_key = "7b2fbb569507ae14141d3de0ff76044999ae3859816024ab9d2978c83bc3d804"


headers = {
    "Accept": "application/json",
    "x-api-key": api_key
}

# loop for the location
for location_id in location_ids:
    
    url = f"https://api.openaq.org/v2/locations/{location_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        # Save the data 
        filename = f"location_{location_id}.json"
        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)
        
        print(f"Data saved to {filename}")
    else:
        print(f'Error for location ID {location_id}: {response.status_code}, {response.text}')


Data saved to location_2163258.json
Data saved to location_6229.json
Data saved to location_270749.json
Data saved to location_7947.json


Sveavägen

Combined all Json files

In [ ]:
import pandas as pd
import json
import os

# Construct the full path dynamically
directory = os.path.join(os.getcwd())
# List to hold all extracted data
all_data = []

# Loop through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        filepath = os.path.join(directory, filename)
        
        with open(filepath, 'r') as json_file:
            data = json.load(json_file)
        
        # Loop through each result in the current file
        for result in data['results']:
            location_name = result['name']
            meter_id = result['id']
            lat = result['coordinates']['latitude']
            lon = result['coordinates']['longitude']

            # Extract the 'day' and 'time' 
            last_updated = result['lastUpdated']
            day, time = last_updated.split('T')
            time = time.split('+')[0]  # Remove the timezone if present

            # Initialize dictionary with default None for each pollutant
            parameter_values = {
                "meter ID": meter_id,
                "location name": location_name,
                "lat": lat,
                "lon": lon,
                "day": day,
                "time": time,
                "pm2.5": None,
                "pm10": None,
                "so2": None,
                "no": None,
                "o3": None
            }

            # Map JSON parameter names to desired column names
            param_map = {
                "pm25": "pm2.5",
                "pm10": "pm10",
                "so2": "so2",
                "no2": "no",
                "o3": "o3"
            }

            # Loop through parameters to find specific pollutants
            for parameter in result['parameters']:
                parameter_name = parameter['parameter']
                last_value = parameter['lastValue']

                # Add the parameter's last value to the dictionary if it matches
                if parameter_name in param_map:
                    parameter_values[param_map[parameter_name]] = last_value

            # Append the dictionary to the all_data list
            all_data.append(parameter_values)

# Convert the combined data to a DataFrame
df = pd.DataFrame(all_data)

# Reorder columns to match desired order
df = df[["meter ID", "location name", "lat", "lon", "day", "time", "pm2.5", "pm10", "so2", "no", "o3"]]

# Display the DataFrame
print(df)

# Optionally, save the DataFrame to a CSV file
df.to_csv('Stockholm_combined.csv', index=False)


historic data

In [31]:
import requests
import json
from IPython.display import display, JSON


country_code = "SE"
city = "Stockholm"
#location_ids = ["2163258", "6229", "270749"]   #Torkel, Svea, sundbyberg
location_ids = ['4318', '154', '4767', '7529', '4593']

# API key
api_key = "7b2fbb569507ae14141d3de0ff76044999ae3859816024ab9d2978c83bc3d804"


headers = {
    "Accept": "application/json",
    "x-api-key": api_key
}

# loop for the location
for location_id in location_ids:
    
    url = f"https://api.openaq.org/v1/measurements?date_from=2023-01-01&limit=10000&page=1&offset=0&sort=desc&radius=1000&location_id={location_id}&order_by=datetime"

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        # Save the data 
        filename = f"location_{location_id}.json"
        with open(filename, 'w') as json_file:
            json.dump(data, json_file, indent=4)
        
        print(f"Data saved to {filename}")
    else:
        print(f'Error for location ID {location_id}: {response.status_code}, {response.text}')


Data saved to location_4318.json
Data saved to location_154.json
Data saved to location_4767.json
Data saved to location_7529.json
Data saved to location_4593.json


In [25]:
import pandas as pd
import json
import os

# Construct the full path dynamically
directory = os.path.join(os.getcwd())
# List to hold all extracted data
all_data = []



# Loop through each file in the directory
param_map = {
    "pm25": "pm2.5",
    "pm10": "pm10",
    "so2": "so2",
    "no2": "no2",
    "o3": "o3",
    "co": "co",
    "no": "no",
    "nox": "nox"
    
}

# Initialize a dictionary to collect data
all_data = {}

# Loop through each JSON file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        filepath = os.path.join(directory, filename)
        
        with open(filepath, 'r') as json_file:
            results = json.load(json_file)['results']
        
        # Loop through each result in the JSON file
        for result in results:
            day, time = result['date']['utc'].split("T")
            location = result['location']
            lat = result['coordinates']['latitude']
            lon = result['coordinates']['longitude']
            param = result['parameter']

            # Use a tuple as a unique key for each location, day, and time
            key = (location, day, time)

            # If the key doesn't exist in all_data, initialize the entry
            if key not in all_data:
                all_data[key] = {
                    "location name": location,
                    "lat": lat,
                    "lon": lon,
                    "day": day,
                    "time": time,
                    "pm2.5": None,
                    "pm10": None,
                    "so2": None,
                    "no2": None,
                    "o3": None,
                    "co": None,
                    "no": None,
                    "nox": None
                    
                }

            # Set the appropriate value in the existing entry
                       # Safeguard against any parameters not in the param_map
            if param in param_map:
                # Set the appropriate value in the existing entry
                all_data[key][param_map[param]] = result['value']
            else:
                print(f"Warning: Parameter '{param}' is not in param_map and was skipped.")
            

# Convert the dictionary to a DataFrame
df = pd.DataFrame(all_data.values())

# Reorder columns
df = df[["location name", "lat", "lon", "day", "time", "pm2.5", "pm10", "so2", "no2", "o3", "co","no", "nox"]]

# Display the DataFrame
print(df)

# Save to CSV
df.to_csv('historic_combined.csv', index=False)


                location name       lat        lon         day  \
0      London Marylebone Road  51.52253  -0.154611  2024-09-04   
1      London Marylebone Road  51.52253  -0.154611  2024-09-04   
2      London Marylebone Road  51.52253  -0.154611  2024-09-04   
3      London Marylebone Road  51.52253  -0.154611  2024-09-04   
4      London Marylebone Road  51.52253  -0.154611  2024-09-04   
...                       ...       ...        ...         ...   
12311                 Arenula  41.89402  12.475368  2024-05-19   
12312                 Arenula  41.89402  12.475368  2024-05-19   
12313                 Arenula  41.89402  12.475368  2024-05-19   
12314                 Arenula  41.89402  12.475368  2024-05-19   
12315                 Arenula  41.89402  12.475368  2024-05-19   

                 time  pm2.5  pm10  so2   no2    o3  co   no   nox  
0      09:00:00+00:00    8.0  12.0  1.0  16.0  19.0 NaN  NaN   NaN  
1      08:00:00+00:00    8.0  13.0  NaN  14.5  26.2 NaN  NaN   NaN  


In [26]:

df_from_csv = pd.read_csv('historic_combined.csv')
print(df_from_csv.head())


            location name       lat       lon         day            time  \
0  London Marylebone Road  51.52253 -0.154611  2024-09-04  09:00:00+00:00   
1  London Marylebone Road  51.52253 -0.154611  2024-09-04  08:00:00+00:00   
2  London Marylebone Road  51.52253 -0.154611  2024-09-04  07:00:00+00:00   
3  London Marylebone Road  51.52253 -0.154611  2024-09-04  06:00:00+00:00   
4  London Marylebone Road  51.52253 -0.154611  2024-09-04  05:00:00+00:00   

   pm2.5  pm10  so2   no2    o3  co  no  nox  
0    8.0  12.0  1.0  16.0  19.0 NaN NaN  NaN  
1    8.0  13.0  NaN  14.5  26.2 NaN NaN  NaN  
2    NaN   NaN  NaN  18.9   NaN NaN NaN  NaN  
3    NaN   NaN  0.7  21.5   NaN NaN NaN  NaN  
4    NaN   NaN  0.7  18.8   NaN NaN NaN  NaN  
